In [29]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# Goal: reproduce nanoGPT tuto from Karpathy
**plan:**
- first implement a word embedder
    - [x] I already got the data(Shakespeare books)
    - [x] read that data
    - [ ] tokenize the data
    - [ ] create an embedder layer 
    - [ ] generator: generate new text: how do I recover a letter based on the output of my network? 

- modify the code to support an attention mechanism

---

as i am coding, I sometimes cheat and see karpathy's code. I tried just to cheat little things like some pythonic way to write something

In [30]:
import random 

# printing first 1000 characters of input.txt
with open("input.txt", 'r') as file:
    text = file.read()
    print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [31]:
# since the idea is to create a character prediction program I guess we should tokenize the characters, not words 

# grab all the different characters in our text
# from karpathy: - sorted - print by joining - vocab_size (should be useful later) 
characters = sorted(list(set(text)))
vocab_size = len(characters)
print(''.join(characters))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [32]:
# I guess this map can account for an embedder 
# from karpathy: -encode -decode, I was doing it for a single character or integer, it should be for a list of such. plus, for the decoder we should output a string, not a list of characters
stoi = {elem:i for i,elem in enumerate(characters)} # stoi: string to index 
itos = {i:elem for i,elem in enumerate(characters)} # 
encode = lambda element: [stoi[elem] for elem in element]
decode = lambda element: ''.join([itos[elem] for elem in element])
print(stoi)
print(encode('hell$$o asd s'))
print(decode(encode('hell$$o asd s')))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
[46, 43, 50, 50, 3, 3, 53, 1, 39, 57, 42, 1, 57]
hell$$o asd s


So Andrej goes on to say that this is heavily based on Bigram.
What is Bigram? 

In [36]:
# - work with batches
# - work with blocks 
import torch

data = torch.tensor(encode(text))
print(data.shape)
print(data[:1000])


torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [40]:
# then we split our data: 90% for train and 10% for validation 

n = int(len(data)*0.9) # 
train_data = data[:n]
val_data = data[n:]
print(train_data.shape)
print(val_data.shape)


torch.Size([1003854])
torch.Size([111540])


In [68]:
# we have to partition our data
# and work in batches
# meaning we will have a collection of `batch_size` random numbers (i.e positions in the text array) and for each of these positions we will
# get a `block`

block_size = 8
batch_size = 4

def get_batch(stage):
    data = train_data if stage=='train' else val_data 
    positions = torch.randint(len(data)-block_size,(batch_size,))
    x = [data[pos:pos+block_size] for pos in positions]
    y = [data[pos+1:pos+block_size+1] for pos in positions]
    # now we need to stack everything
    x = torch.stack(x)
    y = torch.stack(y)
    return x,y


x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    foo = x[:i+1]
    target = y[i]
    print(f'for the context{foo} be have the target {target}')


batch = get_batch('train')
batch[:10]

for the contexttensor([18]) be have the target 47
for the contexttensor([18, 47]) be have the target 56
for the contexttensor([18, 47, 56]) be have the target 57
for the contexttensor([18, 47, 56, 57]) be have the target 58
for the contexttensor([18, 47, 56, 57, 58]) be have the target 1
for the contexttensor([18, 47, 56, 57, 58,  1]) be have the target 15
for the contexttensor([18, 47, 56, 57, 58,  1, 15]) be have the target 47
for the contexttensor([18, 47, 56, 57, 58,  1, 15, 47]) be have the target 58


(tensor([[53, 11,  0, 25, 63,  1, 39, 41],
         [ 8,  0,  0, 28, 30, 21, 26, 15],
         [51, 43, 52, 58,  0, 32, 46, 39],
         [ 6,  1, 61, 46, 53, 57, 43,  1]]),
 tensor([[11,  0, 25, 63,  1, 39, 41, 58],
         [ 0,  0, 28, 30, 21, 26, 15, 17],
         [43, 52, 58,  0, 32, 46, 39, 58],
         [ 1, 61, 46, 53, 57, 43,  1, 52]]))